In [0]:
from delta.tables import *

stage_table_name = "incremental_load.default.orders_stage"
target_table_name = "incremental_load.default.orders_target"

In [0]:
# Read the data from the stage table
stage_df = spark.read.table(stage_table_name)

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.utils import AnalysisException

# Use spark.catalog.tableExists (works in shared mode, but requires full path if Unity Catalog is enabled)
if not spark.catalog.tableExists(target_table_name):
    # Table does not exist → create it
    stage_df.write.format("delta").saveAsTable(target_table_name)

else:
    # Table exists → perform Delta merge
    target_table = DeltaTable.forName(spark, target_table_name)

    merge_condition = "stage.tracking_num = target.tracking_num"

    target_table.alias("target") \
        .merge(stage_df.alias("stage"), merge_condition) \
        .whenMatchedDelete() \
        .execute()

    # Append new records
    stage_df.write.format("delta").mode("append").saveAsTable(target_table_name)
